<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Asset Management

### MVP Portfolio Analysis Class

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## Standardizing Tasks

Topics of interest include:

* Importing, visualizing the data
* `FinancialData` base class
* `MVPPortfolio` portfolio class

## Real Data

**_Historical end-of-day financial time series data._**

Data retrieved vom Refinitiv Eikon for the German DAX 30 constituents.

The data sets:

    http://hilpisch.com/dax_eikon_eod_data.csv
    http://hilpisch.com/dax_eikon_mc_data.csv

## Imports and Data

In [3]:
import math
import cufflinks
#import eikon as ek
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn')
cufflinks.set_config_file(offline=True)
np.set_printoptions(suppress=True, precision=4)
pd.options.display.float_format = '{:.5f}'.format
%config InlineBackend.figure_format = 'svg'

In [4]:
url = 'http://hilpisch.com/dax_eikon_eod_data.csv'

In [5]:
raw = pd.read_csv(url, index_col=0)

In [6]:
raw.iloc[:5, :5]

,1COV.DE,ADSGn.DE,ALVG.DE,BASFn.DE,BAYGn.DE
Date,,,,,
2016-01-04,32.53000,87.54000,156.00000,67.58000,109.28375
2016-01-05,32.15500,87.24000,156.50000,67.20000,109.48057
2016-01-06,31.86000,86.23000,157.00000,66.08000,107.90602
2016-01-07,30.80000,84.89000,153.75000,64.47000,105.15055
2016-01-08,30.09000,83.45000,151.80000,63.68000,102.24747


In [7]:
universe = raw.columns# [-10:]
universe

Index(['1COV.DE', 'ADSGn.DE', 'ALVG.DE', 'BASFn.DE', 'BAYGn.DE', 'BEIG.DE',
       'BMWG.DE', 'CONG.DE', 'DAIGn.DE', 'DB1Gn.DE', 'DBKGn.DE', 'DHER.DE',
       'DPWGn.DE', 'DTEGn.DE', 'DWNG.DE', 'EONGn.DE', 'FMEG.DE', 'FREG.DE',
       'HEIG.DE', 'HNKG_p.DE', 'IFXGn.DE', 'LINI.DE', 'MRCG.DE', 'MTXGn.DE',
       'MUVGn.DE', 'RWEG.DE', 'SAPG.DE', 'SIEGn.DE', 'VNAn.DE', 'VOWG_p.DE'],
      dtype='object')

## Financial Data Class

Methods to be implemented:

    .__init__()
    .retrieve_data()
    .prepare_data()
    .plot_data()
    .plot_mc()
    .plot_corr()

In [8]:
class FinancialData:
    url = 'http://hilpisch.com/dax_eikon_eod_data.csv'
    url_ = 'http://hilpisch.com/dax_eikon_mc_data.csv'
    def __init__(self, universe):
        self.universe = universe
        self.no_assets = len(universe)
        self.retrieve_data()
        self.prepare_data()
    def retrieve_data(self):
        self.raw = pd.read_csv(self.url, index_col=0)
        self.raw_ = pd.read_csv(self.url_, index_col=0)
    def prepare_data(self):
        self.data = self.raw[self.universe]
        self.rets = np.log(self.data / self.data.shift(1))
        self.mc = (self.raw_.T[self.universe]).T
        self.mc['MC%'] = self.mc['MC'].apply(lambda x: x / self.mc['MC'].sum())
    def plot_data(self, cols=None):
        if cols is None:
            cols = self.universe
        self.data[cols].normalize().iplot()
    def plot_mc(self):
        self.mc.sort_values('MC').iplot(kind='pie',
                values='MC', labels='NAME', colorscale='rdylbu')
    def plot_corr(self):
        self.rets.corr().iplot(kind='heatmap', colorscale='reds')

In [9]:
fd = FinancialData(universe)

In [54]:
fd.raw

,1COV.DE,ADSGn.DE,ALVG.DE,BASFn.DE,BAYGn.DE,BEIG.DE,BMWG.DE,CONG.DE,DAIGn.DE,DB1Gn.DE,...,IFXGn.DE,LINI.DE,MRCG.DE,MTXGn.DE,MUVGn.DE,RWEG.DE,SAPG.DE,SIEGn.DE,VNAn.DE,VOWG_p.DE
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,32.53000,87.54000,156.00000,67.58000,109.28375,81.12000,92.25000,215.30000,73.64000,78.03000,...,13.10000,NaN,86.15000,88.02000,178.15000,10.37064,70.58000,77.10187,27.60500,126.40000
2016-01-05,32.15500,87.24000,156.50000,67.20000,109.48057,82.00000,91.82000,216.65000,73.63000,77.79000,...,12.95000,NaN,87.62000,88.30000,177.55000,10.58987,71.43000,77.58866,27.79000,121.40000
2016-01-06,31.86000,86.23000,157.00000,66.08000,107.90602,81.47000,88.78000,209.80000,71.76000,76.82000,...,12.55000,NaN,86.40000,87.43000,178.05000,10.64230,72.05000,77.28217,27.21500,118.90000
2016-01-07,30.80000,84.89000,153.75000,64.47000,105.15055,79.48000,85.44000,204.70000,69.00000,74.99000,...,12.24000,NaN,83.64000,85.52000,174.75000,10.35158,71.34000,75.80377,26.70500,115.00000
2016-01-08,30.09000,83.45000,151.80000,63.68000,102.24747,79.28000,83.44000,202.80000,68.19000,74.39000,...,11.93000,NaN,83.11000,84.64000,172.85000,10.14188,71.05000,75.28092,26.28000,115.10000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-22,49.91000,293.50000,195.22000,63.85000,48.24000,93.08000,72.10000,118.40000,56.64000,137.50000,...,30.83500,208.30000,136.30000,207.30000,237.20000,33.23000,103.62000,114.34000,57.82000,148.90000
2020-12-23,50.36000,293.60000,199.88000,64.69000,48.18000,93.18000,73.38000,121.65000,58.52000,138.20000,...,31.28000,210.60000,136.40000,216.90000,241.80000,33.80000,103.76000,117.00000,58.34000,152.46000
2020-12-28,50.42000,299.30000,202.60000,65.22000,48.58000,94.84000,73.49000,123.05000,58.65000,139.35000,...,31.80500,213.70000,137.55000,214.10000,244.10000,35.02000,105.26000,119.30000,59.16000,152.76000


In [57]:
fd.raw_.T

RIC,1COV.DE,ADSGn.DE,ALVG.DE,BASFn.DE,BAYGn.DE,BEIG.DE,BMWG.DE,CONG.DE,DAIGn.DE,DB1Gn.DE,...,IFXGn.DE,LINI.DE,MRCG.DE,MTXGn.DE,MUVGn.DE,RWEG.DE,SAPG.DE,SIEGn.DE,VNAn.DE,VOWG_p.DE
NAME,COVESTRO/d,ADIDAS N/d,ALLIANZ SE/d,BASF SE/d,BAYER N AG/d,BEIERSDORF/d,BAY MOT WERKE/d,CONTINENTAL AG/d,DAIMLER AG N/d,DT BOERSE N/d,...,INFINEON TECH /d,LINDE PLC/d,MERCK KGAA/d,MTU AERO ENGIN/d,MUENCH. RUECK /d,RWE AG/d,SAP SE/d,SIEMENS N/d,VONOVIA SE/d,VOLKSWAGEN VZ/d
MC,11.14378,56.74613,86.05790,64.56773,51.54349,22.09648,54.78046,23.99323,75.81386,26.16227,...,44.42315,117.65729,17.28256,11.00798,36.02116,20.98735,125.71077,114.82926,31.07853,137.81136
MC%,0.00775,0.03947,0.05986,0.04491,0.03585,0.01537,0.03811,0.01669,0.05274,0.01820,...,0.03090,0.08184,0.01202,0.00766,0.02506,0.01460,0.08745,0.07988,0.02162,0.09586


In [58]:
fd.mc

,NAME,MC,MC%
RIC,,,
1COV.DE,COVESTRO/d,11.14378,0.00775
ADSGn.DE,ADIDAS N/d,56.74613,0.03947
ALVG.DE,ALLIANZ SE/d,86.05790,0.05986
BASFn.DE,BASF SE/d,64.56773,0.04491
BAYGn.DE,BAYER N AG/d,51.54349,0.03585
BEIG.DE,BEIERSDORF/d,22.09648,0.01537
BMWG.DE,BAY MOT WERKE/d,54.78046,0.03811
CONG.DE,CONTINENTAL AG/d,23.99323,0.01669
DAIGn.DE,DAIMLER AG N/d,75.81386,0.05274


In [13]:
fd.plot_data()

In [14]:
fd.plot_mc()

In [15]:
fd.plot_corr()

## MVP Class

Methods to be implemented:

    .__init__()
    .portfolio_return()
    .portfolio_variance()
    .portfolio_volatility()
    .portfolio_sharpe()
    ._set_bounds_constraints()
    ._get_results()
    .minimum_volatility_portfolio()
    .maximum_sharpe_portfolio()
    .plot_weights()
    .plot_performance()

In [16]:
universe = raw.columns[-15:]
universe

Index(['EONGn.DE', 'FMEG.DE', 'FREG.DE', 'HEIG.DE', 'HNKG_p.DE', 'IFXGn.DE',
       'LINI.DE', 'MRCG.DE', 'MTXGn.DE', 'MUVGn.DE', 'RWEG.DE', 'SAPG.DE',
       'SIEGn.DE', 'VNAn.DE', 'VOWG_p.DE'],
      dtype='object')

In [17]:
from scipy.optimize import minimize

In [18]:
class MVPPortfolio(FinancialData):
    def __init__(self, universe):
        super().__init__(universe)
        self.equal_weights = self.no_assets * [1 / self.no_assets]
        self.mc_weights = self.mc['MC%'].values
    def portfolio_return(self, weights, days=252):
        return np.dot(self.rets.mean(), weights) * days
    def portfolio_variance(self, weights, days=252):
        return np.dot(weights, np.dot(self.rets.cov(), weights)) * days
    def portfolio_volatility(self, weights, days=252):
        return math.sqrt(self.portfolio_variance(weights, days))
    def portfolio_sharpe(self, weights, days=252):
        sharpe = (self.portfolio_return(weights, days) /
                  self.portfolio_volatility(weights, days))
        return sharpe
    def _set_bounds_constraints(self, bnds, cons):
        if bnds is None:
            self.bnds = self.no_assets * [(0, 1)]
        else:
            self.bnds = bnds
        if cons is None:
            self.cons = {'type': 'eq', 'fun': lambda weights: weights.sum() - 1}
        else:
            self.cons = cons
    def _get_results(self, opt, kind):
        ret = self.portfolio_return(opt['x'])
        vol = self.portfolio_volatility(opt['x'])
        sharpe = self.portfolio_sharpe(opt['x'])
        weights = pd.DataFrame(opt['x'], index=self.universe, columns=['weights',])
        res = {'kind': kind, 'weights': weights.round(7), 'return': ret,
               'volatility': vol, 'sharpe': sharpe}
        return res
    def minimum_volatility_portfolio(self, bnds=None, cons=None):
        self._set_bounds_constraints(bnds, cons)
        opt = minimize(self.portfolio_volatility, self.equal_weights,
                      bounds=self.bnds, constraints=self.cons)
        self.results = self._get_results(opt, 'Minimum Volatility')
        return self.results
    def maximum_sharpe_portfolio(self, bnds=None, cons=None):
        self._set_bounds_constraints(bnds, cons)
        tf = lambda weights: -self.portfolio_sharpe(weights)
        opt = minimize(tf, self.equal_weights, bounds=self.bnds,
                       constraints=self.cons)
        self.results = self._get_results(opt, 'Maximum Sharpe')
        return self.results
    def plot_weights(self, kind='pie'):
        if kind == 'pie':
            nonzero = self.results['weights'] > 0
            to_plot = self.results['weights'][nonzero['weights']].copy()
            to_plot['names'] = to_plot.index
            to_plot.iplot(kind='pie', values='weights',
                          labels='names', colorscale='rdylbu',
                          title='Optimal Weights | ' + self.results['kind'])
        else:
            self.results['weights'].iplot(kind='bar',
                    title='Optimal Weights | ' + self.results['kind'])
    def plot_performance(self):
        perf = (self.results['return'], self.results['volatility'], self.results['sharpe'])
        index = ['return', 'volatility', 'sharpe']
        to_plot = pd.DataFrame(perf, index=index, columns=['metrics',])
        to_plot.iplot(kind='bar', title='Performance Metrics  | ' + self.results['kind'])

In [19]:
mvp = MVPPortfolio(universe)

In [20]:
# mvp.equal_weights

In [21]:
mvp.portfolio_return(mvp.equal_weights)

0.07664236035558741

In [22]:
mvp.portfolio_volatility(mvp.equal_weights)

0.2011121924535967

In [23]:
mvp.portfolio_sharpe(mvp.equal_weights)

0.38109256042878337

In [24]:
mvp.mc_weights

array([0.0298, 0.023 , 0.0262, 0.0185, 0.048 , 0.0578, 0.1531, 0.0225,
       0.0143, 0.0469, 0.0273, 0.1636, 0.1494, 0.0404, 0.1793])

In [25]:
mvp.portfolio_return(mvp.mc_weights)

0.08603112261355028

In [26]:
mvp.portfolio_volatility(mvp.mc_weights)

0.2204678726189514

In [27]:
mvp.portfolio_sharpe(mvp.mc_weights)

0.3902206774691536

In [28]:
res = mvp.minimum_volatility_portfolio()

In [29]:
res['weights']

,weights
EONGn.DE,0.08355
FMEG.DE,0.13602
FREG.DE,0.00000
HEIG.DE,0.00000
HNKG_p.DE,0.27670
IFXGn.DE,0.00000
LINI.DE,0.00000
MRCG.DE,0.15992
MTXGn.DE,0.00000
MUVGn.DE,0.04857


In [30]:
res['return']  # expected return

0.0589625081751955

In [31]:
res = mvp.maximum_sharpe_portfolio(bnds=None)

In [32]:
res['weights']

,weights
EONGn.DE,0.00000
FMEG.DE,0.00000
FREG.DE,0.00000
HEIG.DE,0.00000
HNKG_p.DE,0.00000
IFXGn.DE,0.00269
LINI.DE,0.15724
MRCG.DE,0.00000
MTXGn.DE,0.07399
MUVGn.DE,0.00000


In [33]:
res = mvp.maximum_sharpe_portfolio(bnds=[])

In [34]:
res['weights']

,weights
EONGn.DE,-1.54313
FMEG.DE,0.08217
FREG.DE,-2.35765
HEIG.DE,-2.02932
HNKG_p.DE,-2.43228
IFXGn.DE,0.84964
LINI.DE,1.91811
MRCG.DE,0.90912
MTXGn.DE,1.26214
MUVGn.DE,-0.30537


### Minimum Volatility

In [35]:
res = mvp.minimum_volatility_portfolio()

In [36]:
res['weights']

,weights
EONGn.DE,0.08355
FMEG.DE,0.13602
FREG.DE,0.00000
HEIG.DE,0.00000
HNKG_p.DE,0.27670
IFXGn.DE,0.00000
LINI.DE,0.00000
MRCG.DE,0.15992
MTXGn.DE,0.00000
MUVGn.DE,0.04857


In [37]:
mvp.plot_weights()

In [38]:
mvp.plot_performance()

### Maximum Sharpe Ratio 

In [39]:
res = mvp.maximum_sharpe_portfolio()

In [40]:
res['weights']

,weights
EONGn.DE,0.00000
FMEG.DE,0.00000
FREG.DE,0.00000
HEIG.DE,0.00000
HNKG_p.DE,0.00000
IFXGn.DE,0.00269
LINI.DE,0.15724
MRCG.DE,0.00000
MTXGn.DE,0.07399
MUVGn.DE,0.00000


In [41]:
mvp.plot_weights()

In [42]:
mvp.plot_weights(kind='bar')

### Bounded Maximum Sharpe

In [43]:
bnds = mvp.no_assets * [(0.01, 0.4),]

In [44]:
res = mvp.maximum_sharpe_portfolio(bnds=bnds)

In [45]:
res['weights']

,weights
EONGn.DE,0.01000
FMEG.DE,0.01000
FREG.DE,0.01000
HEIG.DE,0.01000
HNKG_p.DE,0.01000
IFXGn.DE,0.01000
LINI.DE,0.12068
MRCG.DE,0.01000
MTXGn.DE,0.05831
MUVGn.DE,0.01000


In [46]:
mvp.plot_weights()

In [47]:
mvp.plot_weights(kind='bar')

### Short Sales Allowed

In [48]:
bnds = mvp.no_assets * [(-1, 1),]

In [49]:
cons = {}

In [50]:
res = mvp.maximum_sharpe_portfolio(bnds=bnds, cons=cons)

In [51]:
res['weights']

,weights
EONGn.DE,-0.52594
FMEG.DE,0.02872
FREG.DE,-0.80638
HEIG.DE,-0.69256
HNKG_p.DE,-0.83139
IFXGn.DE,0.28944
LINI.DE,0.65548
MRCG.DE,0.31052
MTXGn.DE,0.43118
MUVGn.DE,-0.10257


In [52]:
mvp.plot_weights(kind='bar')

In [53]:
mvp.plot_performance()

## Exercise

**Use the `MVPPortfolio` class and adjust it in a way that it becomes a `BL92Portfolio` class.**

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="30%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>